In [ ]:
import pandas
import re

In [ ]:
tableau_complet = pandas.read_csv("../donnees-welcome-guide/OKiS1cuixs-codelico.csv")
tableau_complet["Code postal"] = tableau_complet["Code postal"].astype("object")

In [ ]:
colonne_ordre = ["Nom", "adresse"]

In [ ]:
tableau_complet_ordonne = tableau_complet.sort_values(colonne_ordre)

In [ ]:
tableau_complet_ordonne.to_csv("../donnees-welcome-guide/ordonnees.csv")

## Tableau des organisations et adresses

In [ ]:
tableau_complet_ordonne.columns

Sélection des colonnes correspondant aux orgas sans les services (sans "Texte complémentaire Codelico")

In [ ]:
tableau_organisations_colonnes = tableau_complet_ordonne[['Nom', 'adresse', 'Code postal', 'Ville', 'Téléphone', 'Email',
       'Site web', 'Texte complémentaire Codelico', 'Liste de catégories', 'Type ','Pays', 'urlImg']]

### Sélection des organisations dupliquées

Pour chaque organisation dupliquée (même nom, même adresse), stocker les infos et fusionner la liste de catégories et fusionner les services.

Obtention de la liste des organisations dupliquées

In [ ]:
masque_orga_dupliquee = tableau_organisations_colonnes.duplicated(["Nom", "adresse"], keep="first")
nom_organisations_dupliquees = tableau_organisations_colonnes[masque_orga_dupliquee].Nom

Concaténation des services dans une seule cellule (idem pour les catégories).

In [ ]:
valeurs_pour_orgas_dedupliquees = pandas.DataFrame({}, columns=tableau_complet.columns)

liste_d_index_dupliques_et_geres = []

for organisation in nom_organisations_dupliquees:
    
    # Sélection des lignes correspondant à une seule organisation (dont on sait qu'elle est dupliquée)
    masque_organisation_est_dupliquee = tableau_organisations_colonnes.Nom == organisation
    tableau_pour_orga_dupliquee = tableau_organisations_colonnes[masque_organisation_est_dupliquee]
    
    # Conservation en mémoire des lignes qui sont dans le tableau des organisations dupliquées
    for index in tableau_pour_orga_dupliquee.index.values:
        liste_d_index_dupliques_et_geres.append(index)
        

    # Concaténation de la liste de services
    liste_de_services = []
    for service in tableau_pour_orga_dupliquee["Texte complémentaire Codelico"]:                
        liste_de_services.append(service + "\n\n\n")
    
    # Concaténation de la liste de catégories
    liste_de_categories = []
    liste_de_categories_dupliquees = []
    for categories in tableau_pour_orga_dupliquee["Liste de catégories"]:
        liste_de_categories_pour_cette_orga_a_nettoyer = categories.split(", ")
        
        liste_de_categories_dupliquees_pour_cette_orga = []
        for categorie_a_nettoyer in liste_de_categories_pour_cette_orga_a_nettoyer:
            
            categorie = re.sub(r" *$", "", re.sub(r"^ *", "", categorie_a_nettoyer))
        
            liste_de_categories_dupliquees_pour_cette_orga.append(categorie)
              
        liste_de_categories_dupliquees += liste_de_categories_dupliquees_pour_cette_orga
        
    liste_de_categories = list(set(liste_de_categories))
    
    
    
    # Pour une orga dupliquée, sélection d'une seule ligne, peu importe les valeurs de service et catégories
    valeurs_pour_orga = tableau_pour_orga_dupliquee.head(1)
    
    # Stockage de la liste des services et de la liste des catégories dans cette ligne
    valeurs_pour_orga["Liste de catégories"] = ",".join(liste_de_categories)
    valeurs_pour_orga["Texte complémentaire Codelico"] = ",".join(liste_de_services)
    
    # Ajout de la ligne avec les services concaténés et catégories concaténées dans un tableau
    liste_de_valeurs_pour_orgas_dedupliquees.append(valeurs_pour_orga)

valeurs_pour_orgas_dedupliquees = pandas.concat(liste_de_valeurs_pour_orgas_dedupliquees).drop_duplicates()

In [ ]:
valeurs_pour_orgas_dedupliquees.head()

### Sélection des lignes des organisations non dupliquées

Il s'agit de toutes les autres lignes que celles dont l'indice a été vu dans la gestion des organisations dupliquées.

In [ ]:
valeurs_pour_orgas_non_dupliquees = tableau_organisations_colonnes.drop(liste_d_index_dupliques_et_geres)

### Création d'un tableau avec les organisations non dupliquées et les organisations dupliquées

In [ ]:
valeurs_pour_orgas = valeurs_pour_orgas_dedupliquees.append(valeurs_pour_orgas_non_dupliquees)

In [ ]:
valeurs_pour_orgas.head()

Ce tableau contient une ligne par organisation, avec les services placés dans une seule cellule. Idem pour les catégories.

In [ ]:
valeurs_pour_orgas.to_csv("../donnees-welcome-guide/une-organisation-services-concatenes.csv", sep=";", index=False)

## Import dans Communecter

Import via admin public avec mon compte.

![](../images/config-admin-public-codelicoconcatene20190622.png)

In [ ]:
clef = "codelicoconcatene20190622"

Dans un premier temps, l'import ne fonctionne pas.